### Open a browser and set the cookies from a JSON file

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time,json, random, re, datetime
import pandas as pd

def setCookiesFromJson():
    with open('cookies.json', 'r', newline='') as inputdata:
        cookies = json.load(inputdata)
    for cookie in cookies: #works only after driver.get
        driver.add_cookie(cookie)
    driver.refresh() # to load cookies

service = Service(executable_path="chromedriver.exe")
chrome_options = Options()
chrome_options.add_argument("--disable-search-engine-choice-screen")
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging']) #disable error logging
driver = webdriver.Chrome(service=service, options=chrome_options)

# base_url = "https://theprotocol.it/filtry/python;t/ai-ml;sp"
# base_url = "https://theprotocol.it/filtry/python;t/ai-ml;sp/bialystok;wp"
# base_url = "https://theprotocol.it/filtry/sql,python,javascript;t/junior,trainee,assistant;p"
# base_url = "https://theprotocol.it/filtry/helpdesk;sp/warszawa,bialystok;wp/zdalna,hybrydowa,stacjonarna;rw/"
base_url = "https://theprotocol.it/filtry/trainee,assistant,junior;p"
driver.get(base_url)
setCookiesFromJson()

### Fetch the URLs from all the pages

In [3]:
def anyOffersOnTheList():
    try:
        driver.find_element(By.CSS_SELECTOR, '#main-offers-listing > div.hfenof > div.t2re51w > div')
        return False
    except:
        return True
    
offers_urls = []

def fetchOffersUrlsFromSinglePage():
    offersContainer = driver.find_element("xpath", '//*[@id="main-offers-listing"]/div[1]/div')
    offers = offersContainer.find_elements(By.CLASS_NAME, 'a4pzt2q ')
    # offers = offersContainer.find_elements(By.CSS_SELECTOR, '#offer-title') #also works
    # print('\t'+ str(len(offers)) + ' offers:')
    for offer in offers:
        offers_urls.append(offer.get_property("href"))

page = 1 #theprotocol enumerates pages starting from 1
while True: # because not sure how many pages are there
    site = driver.get(base_url + "?pageNumber=" + str(page))
    if not anyOffersOnTheList():
        print('fetched ' + str(len(offers_urls)) + ' offer urls in total')
        break # break if no results
    else:
        time.sleep(random.uniform(0.5, 1)) #humanize
        fetchOffersUrlsFromSinglePage()
        print('page ' + str(page) + ' urls fetched')
        page += 1

page 1 urls fetched
page 2 urls fetched
page 3 urls fetched
page 4 urls fetched
page 5 urls fetched
page 6 urls fetched
page 7 urls fetched
page 8 urls fetched
fetched 355 offer urls in total


### Analyse offer functions

In [4]:
def offerNotFound():
    try:
        driver.find_element("xpath", '//*[@data-test="text-offerNotFound"]')
        return True
    except:
        return False
    
def getOfferDetails():
    #JOB TITLE
    try:
        jobTitle = driver.find_element(By.XPATH, '//*[@data-test="text-offerTitle"]') # this element should always exist
        jobTitle = jobTitle.text
    except:
        jobTitle = None
    
    #SALARY
    try:
        salaryContainer = driver.find_element(By.XPATH, '//*[@data-test="section-contract"]') # this element should always exist
        salaryAndContract = salaryContainer.text
        # print(salaryAndContract  + '\n')
    except:
        salaryAndContract = None
    
    salaryMinAndMax = [None, None]
    if salaryAndContract:
        try: #to recalculate salary to [PLN/month net] #PLN=only unit on protocol?
            grossToNetMultiplier = 0.7
            hoursPerMonthInFullTimeJob = 168
            lines = salaryAndContract.splitlines()
            if len(lines) >= 3: #should be 2-3 tho
                lines[0] = lines[0].replace(" ", "") #remove spaces
                lines[0] = re.sub(r",\d{1,2}", '', lines[0]) #removes dash and /d x(1-2)  (needed when salary as 123,45)
                salaryMinAndMax = re.findall(r"\d+", lines[0]) #r = raw
                # print(salaryMinAndMax.split(',', 1)[0])
                # salaryUnit = re.findall(r"[^\d–-]", lines[0]) #[exclude digits and –/-]
                # salaryUnit = ''.join(salaryUnit) #join list elements
                if re.findall("brutto", lines[1]) or re.findall("gross", lines[1]): # gross -> net
                    salaryMinAndMax = [(float(elmnt) * grossToNetMultiplier) for elmnt in salaryMinAndMax]
                    # print(salaryMinAndMax)
                if re.findall("godz", lines[1]) or re.findall("hr.", lines[1]): # hr -> month
                    salaryMinAndMax = [(float(elmnt) * hoursPerMonthInFullTimeJob) for elmnt in salaryMinAndMax] #possible input float/str

                salaryMinAndMax = [int(elmnt) for elmnt in salaryMinAndMax] # to ints
        except:
            pass    # salaryMinAndMax = [None, None]

    # EMPLOYER
    try:
        employerElement = driver.find_element("xpath", '//*[@data-test="anchor-company-link"]') # this element should always exist
        employer = employerElement.text + ' ' + employerElement.get_property("href")
    except:
        employer = None
    # print(employer  + '\n')
    
    #WORKFROM, EXP, VALIDTO, LOCATION - "PARAMETERS"
    workModes, positionLevels, offerValidTo, location = '', '', '', ''
    parametersContainer = driver.find_element(By.CLASS_NAME, "c21kfgf")
    parameters = parametersContainer.find_elements(By.CLASS_NAME, "s1bu9jax")
    for param in parameters:
        paramType = param.get_attribute("data-test") #element description
        match paramType:
            case "section-workModes":
                workModes = param.text
            case "section-positionLevels":
                positionLevels = param.text
            case "section-offerValidTo":
                offerValidTo = param.text
            case "section-workplace":
                location = param.text
                try: #to find and click 'more locations' button then fetch what's inside
                    moreLocations = driver.find_element("xpath", '//*[@data-test="button-locationPicker"]')
                    moreLocations.click()
                    # time.sleep(0.05) #probably necessary
                    locations = moreLocations.find_element("xpath", '//*[@data-test="modal-locations"]')
                    location = locations.text
                except:
                    pass #leave location as it was
    # print(workModes + '\n\n' + positionLevels + '\n\n' +  offerValidTo + '\n\n' +  location + '\n')

    # # # TECHSTACK
    descriptionsContainer = driver.find_element(By.CSS_SELECTOR, '#TECHNOLOGY_AND_POSITION')

    techstack = descriptionsContainer.find_elements(By.CLASS_NAME, "c1fj2x2p")
    techstackExpected = None
    techstackOptional = None
    for group in techstack:
        if group.text[0:8] == 'EXPECTED' or group.text[0:8] == 'WYMAGANE':
            techstackExpected = group.text[9:]
        elif group.text[0:8] == 'OPTIONAL': #never saw polish version yet
            techstackOptional = group.text[9:]
    # print(techstackExpected + '\n\n' + techstackOptional + '\n')

    #RESPONSIBILITIES
    try:
        try:
            responsibilities = descriptionsContainer.find_element("xpath", '//*[@data-test="section-responsibilities"]/ul').text #/only ul elements
        except:
            responsibilities = descriptionsContainer.find_element("xpath", '//*[@data-test="section-responsibilities"]').text #/if it's a single entry
    except:
        responsibilities = None
        # print('RESPONSIBILITIES:\n' + str(responsibilities) + '\n' + driver.current_url)

    #REQUIREMENTS
    try:
        try:
            requirements = descriptionsContainer.find_element("xpath", '//*[@data-test="section-requirements"]/ul').text
        except:
            requirements = descriptionsContainer.find_element("xpath", '//*[@data-test="section-requirements"]').text #/if it's a single entry
    except:
        requirements = None
        # print('REQUIREMENTS:\n' + str(requirements) + '\n' + driver.current_url)


    #OPTIONAL REQUIREMENTS
    try:
        optionalRequirementsContainer = descriptionsContainer.find_elements("xpath", '//*[@data-test="section-requirements-optional"]/li')
        if len(optionalRequirementsContainer) > 0:
            optionalRequirements = ''
            for optionalRequirement in optionalRequirementsContainer:
                optionalRequirements += optionalRequirement.text + '\n'
        elif len(optionalRequirementsContainer) <= 0:
            try:
                optionalRequirements = descriptionsContainer.find_element("xpath", '//*[@data-test="section-requirements-optional"]').text
            except:
                optionalRequirements = None
                # print('OPTIONAL:\n' + str(optionalRequirements) + '\n' + driver.current_url)        
    except:
        optionalRequirements = None
    # print('OPTIONAL:\n' + str(optionalRequirements) + '\n' + driver.current_url)
    datetimeNow = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    return [datetimeNow, datetimeNow, driver.current_url, jobTitle, salaryAndContract, salaryMinAndMax[0], salaryMinAndMax[1], employer, workModes, positionLevels, offerValidTo, location, techstackExpected, techstackOptional, responsibilities, requirements, optionalRequirements]

# driver.get('https://theprotocol.it/szczegoly/praca/informatyk-zamosc-strefowa-10,oferta,cd300000-23ab-a26d-445c-08dce14a2562?s=8321028996&searchId=243e7f60-84d9-11ef-8246-7b66699012e8')
# getOfferDetails()

### Database management functions

In [5]:
# print(resultsDataFrame.employer)
# # resultsDataFrame.to_sql('offers', 'resultsDf.db') #alchemy needed
import sqlite3

tableName = 'test4' #not needed as an argument

class database():
    def createTableIfNotExists(): #if not exists
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("CREATE TABLE IF NOT EXISTS " + tableName + """ (
                    datetimeFirst TEXT,
                    datetimeLast TEXT,
                    url TEXT,
                    title TEXT, 
                    salaryAndContract TEXT,
                    salaryMin INT,
                    salaryMax INT,
                    employer TEXT,
                    workModes TEXT,
                    positionLevels TEXT,
                    offerValidTo TEXT,
                    location TEXT,
                    techstackExpected TEXT,
                    techstackOptional TEXT,
                    responsibilities TEXT,
                    requirements TEXT,
                    optionalRequirements TEXT);""")
        connection.commit()
        cursor.close()
        connection.close()

    def selectAll():
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM" + tableName +";")
        connection.commit()
        print(cursor.fetchall())
        cursor.close()
        connection.close()

    def executeQuery(query):
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        # print(cursor.fetchall())
        cursor.close()
        connection.close()
    
    def recordFound(url):
        urlPartToCompare = re.split("[?]s=", url)[0] #split on '?s=' because after that it's only session related stuff. If no pattern found url unchanged
        # print(urlPartToCompare)
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("SELECT datetimeFirst FROM " + tableName + " WHERE url LIKE ('%" + urlPartToCompare + "%');")
        connection.commit()
        result = cursor.fetchall()
        cursor.close()
        connection.close()
        if len(result) >0:
            return True
        else:
            return False

    def insertRecord(dictionary):
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("INSERT INTO " + tableName + " VALUES (:datetimeFirst, :datetimeLast, :url, :title, :salaryAndContract, :salaryMin, :salaryMax, :employer, :workModes, :positionLevels, :offerValidTo, :location, :techstackExpected, :techstackOptional, :responsibilities, :requirements, :optionalRequirements)", dictionary)
        connection.commit()
        cursor.close()
        connection.close()

    def updateDatetimeLast(url):
        urlPartToCompare = re.split("[?]s=", url)[0] #split on '?s=' because after that it's only session related stuff. If no pattern found url unchanged
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("UPDATE " + tableName + " SET datetimeLast = '" + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")) + "'  WHERE url LIKE ('%" + urlPartToCompare + "%');")
        # cursor.execute("SELECT datetimeLast FROM " + tableName + " WHERE url LIKE ('%" + urlPartToCompare + "%');")
        connection.commit()
        cursor.close()
        connection.close()
    
    def countAllRecords():
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        cursor.execute("SELECT COUNT (*) FROM " + tableName +";")
        connection.commit()
        resultTuple = cursor.fetchall()[0]
        (count,) = resultTuple #unpacking tuple
        cursor.close()
        connection.close()
        return str(count)

    def queryToDataframe(fullQuery):
        connection = sqlite3.connect('results.db')
        cursor = connection.cursor()
        # df = pd.read_sql("SELECT datetimeFirst, datetimeLast FROM " +tableName+ ";", con=connection)
        df = pd.read_sql(fullQuery, con=connection)
        connection.commit()
        # print(cursor.fetchall())
        # print('\n'+str(len(cursor.fetchall())) + ' records found')
        cursor.close()
        connection.close()
        return df
    
database.createTableIfNotExists()
# database.queryBuilder(False, ['SQL'], ['javascript', 'c++'])
database.countAllRecords()
# database.executeQuery("DROP TABLE" + tableName)

'207'

### Scrapping to database

In [ ]:
columnsAll = ['datetimeFirst', 'datetimeLast', 'url', 'title', 'salaryAndContract', 'salaryMin', 'salaryMax', 'employer', 'workModes', 'positionLevels', 'offerValidTo', 'location', 'techstackExpected', 'techstackOptional', 'responsibilities', 'requirements', 'optionalRequirements'] # move out of global scope later

import numpy as np

def scrapToDatabase():
    timeDeltas = []
    inserts = 0
    updates = 0
    print(database.countAllRecords() + ' records before run')
    # for i in range (0,2):
    for i in range (len(offers_urls)):
        driver.get(offers_urls[i])
        if not offerNotFound():
            resultsList = getOfferDetails()
            outputDictionary = {}
            for column, offerDetail in zip(columnsAll, resultsList):
                outputDictionary[column] = offerDetail #combine 2 lists into 1 dictionary
            # before = time.time()
            if database.recordFound(driver.current_url):
                database.updateDatetimeLast(driver.current_url)
                # print(driver.current_url)
                updates += 1
            else:
                database.insertRecord(outputDictionary) # insert into databas
                inserts += 1
                # print('insert')
            # timeDeltas.append(time.time() - before)
            #ending here and starting in an above for/zip loop it takes ~(1/100)s - good enough
            print (str(i+1) + '/' + str(len(offers_urls)) + ' done')
        else:
            print('OFFER NOT FOUND: ' +  driver.current_url)
        time.sleep(random.uniform(0.35,0.85)) #Humanize requests frequency
    # print(np.mean(timeDeltas))
    print(str(inserts) + ' inserts | ' + str(updates) + ' updates')

scrapToDatabase()

In [15]:
from flask import Flask, render_template, request
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource
from bokeh.embed import components
from bokeh.layouts import column

def makeBokehPlot(): #Only offers with specified salary?
    plot = figure(title="Title", x_axis_label='X', y_axis_label='Y', height = 400, sizing_mode='stretch_width')
    plot.line([1, 2, 3, 4, 5], [16, 7, 2, 4, 5], line_width=2)
    # plot.line([16, 7, 2, 4, 5], [4, 2, 3, 4, 5], line_width=2)
    return plot

def makeBokehTable(dataframe):
    source = ColumnDataSource(dataframe)
    columns = [(TableColumn(field=i, title=i)) for i in dataframe.columns]
    table = DataTable(source=source, columns=columns, height = 800, editable=True, sizing_mode="stretch_width")
    return table

app = Flask(__name__)

@app.route('/', methods=['GET'])
def results():
    layout = column(makeBokehPlot(), makeBokehTable())
    # Get the script and div elements to embed in the template
    script, div = components(layout)
    # Render the template, passing the components
    return render_template("app.html", script=script, div=div, cdn_js=CDN.js_files, cdn_css=CDN.css_files)

@app.route('/form', methods=['GET', 'POST'])
def form():
    if request.method == 'GET':
        return render_template("form.html", columnsAll=columnsAll)
    
    elif request.method == 'POST':
        def makeFormOutputDictionary():
            formOutputDict = {}
            for column in columnsAll:
                rowDictionary = {'show': False, 'necessary': None, 'forbidden': None, 'above': None, 'below': None}
                #show column
                if request.form.get(column+'Show', False): #if not found assign False. Found only if form field not empty
                    rowDictionary['show'] = True
                #necessary phrase
                if request.form.get(column+'Necessary', False):
                    phraseNecessary = request.form.get(column+'Necessary')
                    phraseNecessary = phraseNecessary.split(", ")
                    rowDictionary['necessary'] = phraseNecessary
                #forbidden phrase
                if request.form.get(column+'Forbidden', False):
                    phraseForbidden = request.form.get(column+'Forbidden')
                    phraseForbidden = phraseForbidden.split(", ")
                    rowDictionary['forbidden'] = phraseForbidden
                #above
                if request.form.get(column+'Above', False):
                    rowDictionary['above'] = request.form.get(column+'Above')
                    # print('found ' + column+'Above') #
                #below
                if request.form.get(column+'Below', False):
                    rowDictionary['below'] = request.form.get(column+'Below')
                    # print('found ' + column+'Below') #
                formOutputDict[column] = rowDictionary #append row with column name as a key
            return formOutputDict
        
        def queryBuilder(formDictionary):
            querySelectPart = "SELECT "
            queryMainPart = ""
            for columnName in formDictionary.keys():
                currentColumnDictionary = formDictionary[columnName].items()
                for key, value in currentColumnDictionary:
                    # SELECT STATEMENT APPENDING
                    if key == 'show' and value:
                        querySelectPart += columnName + ', '
                    #ABOVE & BELOW 
                    if key == 'above' and value:
                        queryMainPart += "\nAND "+columnName+" > '"+value+"'"
                    if key == 'below' and value:
                        queryMainPart += "\nAND "+columnName+" < '"+value+"'"
                    #NECESSARY/FORBIDDEN PHRASES
                    if key == 'necessary' and value: # if list not empty
                        for necessaryPhrase in value:
                            queryMainPart += "\nAND "+columnName+" LIKE ('%"+necessaryPhrase+"%')"
                    if key == "forbidden" and value:
                        for forbiddenPhrase in value:
                            queryMainPart += "\nAND "+columnName+" NOT LIKE ('%"+forbiddenPhrase+"%')"
            queryMainPart += 'ORDER BY salaryMin'

            querySelectPart = querySelectPart[:-2]#remove ", " from the end
            querySelectPart += " FROM "+tableName+" WHERE 1=1" # to append only ANDs
            query = querySelectPart + queryMainPart
            queryPlotSelectPart = "SELECT datetimeFirst, datetimeLast, url, title, salaryMin, salaryMax FROM "+tableName+" WHERE 1=1"#2nd query - always select datetimes and salaries for plotting
            # print(queryPlotSelectPart + queryMainPart)
            # lines[0] = re.sub(r",\d{1,2}", '', lines[0])
            return query
        
        global df #delete later
        df = database.queryToDataframe(queryBuilder(makeFormOutputDictionary()))
        # df.to_csv('results.csv', sep=',', encoding='utf-8-sig', index=True, header=True) #export to CSV
        # print(len(df))
        

        
        layout = column(makeBokehPlot(), makeBokehTable(df))
        # Get the script and div elements to embed in the template
        script, div = components(layout)
        # return makeFormOutputDictionary()
        return render_template("app.html", script=script, div=div, cdn_js=CDN.js_files, cdn_css=CDN.css_files)

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)#JUPYTER

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Oct/2024 20:37:28] "GET /form HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2024 20:39:08] "GET /form HTTP/1.1" 200 -


In [53]:
# # ADJUST QUERY BUILDER:
# def queryBuilder(formOutput):
#     fullQuery = "SELECT datetimeLast, techstackExpected FROM " + tableName + " WHERE 1=1" #Includes WHERE to always append a line starting with AND
#     if onlyLastSeenLessThan24hAgo:
#         fullQuery += "\nAND (JULIANDAY(strftime('%Y-%m-%d %H:%M:%S', DATETIME('now', 'localtime'))) - JULIANDAY(datetimeLast)) * 24 < 24" 
#     if keywordsNecessary:
#         for keyword in keywordsNecessary:
#             fullQuery += "\nAND (techstackExpected LIKE ('%"+keyword+"%') OR requirements LIKE ('%"+keyword+"%'))"
#     if keywordsUnacceptable:
#         for keyword in keywordsUnacceptable:
#             fullQuery += "\nAND techstackExpected NOT LIKE ('%"+keyword+"%') AND requirements NOT LIKE ('%"+keyword+"%')"
#     fullQuery += ';'
#     print(fullQuery)

## TODO:
# waluty -> zł? chyba nie ma innej waluty na stronie
# elif group.text[0:8] == 'OPTIONAL': #never saw polish version yet
# wykresy posortowane po salaryMin/Max
# wykresy po zliczeniu słów?
# paramsy takie jak %VAT do ustawienia
# download csv table?
# form OR operator (/)?
# query ORDER BY part potrzebne?


# list = df[['datetimeFirst', 'datetimeLast']].values.tolist()
# print(list)

ls = pd.to_datetime(df["datetimeFirst"])
ls = pd.to_datetime(df["datetimeLast"])-pd.to_datetime(df["datetimeFirst"])
print(ls)


0      4 days 22:19:15
1      4 days 22:19:16
2     17 days 00:51:08
3     12 days 23:11:22
4     12 days 23:11:26
            ...       
202    0 days 00:00:00
203    0 days 00:00:00
204    0 days 00:00:00
205    0 days 00:00:00
206    0 days 00:00:00
Length: 207, dtype: timedelta64[ns]
